# Running Processes

In [1]:
command = `link README.rst foo.txt`

`link README.rst foo.txt`

What is it? It's glorious!

In [2]:
typeof(command)

Cmd

It's Julia's `Cmd` literal, and it's a thing of beauty. What has it done? Nothing.

Command literals, though they look the same, are not like process substitution in Perl, Ruby or Bash in that they execute a command and return the output as a string. They are something so much better. The create a `Cmd` instance which contains the arguments and some other information, and that object can be sent to various different functions to be executed in different ways. [The documentation](https://docs.julialang.org/en/v1/manual/running-external-programs/) gives a good description of how to use these little marvels, so I'll just cover a few simple cases here and explain what makes these so great.

The simplest thing you can do, and the thing you need most often, is simply to run the command.

In [3]:
filename = "foo.txt"
run(`link README.rst $filename`)

Process(`link README.rst foo.txt`, ProcessExited(0))

In [4]:
run(`ls -lh $filename`)

-rw-r--r-- 2 ninjaaron ninjaaron 7,7K Jan 26 14:50 foo.txt


Process(`ls -lh foo.txt`, ProcessExited(0))

What actually happened here? Obviously we ran the `link` executable and the `ls` executable on the local system, but maybe not in the way you'd expect in other languages, the default methods for running commands _generally_ create a subshell and execute your input there. In Julia, commands never get a shell. As far as I know, the only way to give a command a shell would be to do so explicitely, something like `bash -c echo "my injection vulnerability"`, but you really don't need a shell, so that's fine. What Julia's command literals do is pass the string to parser for a shell-like mini-language, which converts the command into a vector of strings which will ultimately be handed to one of the OS's `exec` familiy of functions--on \*nix. I don't know how these things happen on Windows.

The result is that running commands in Julia is safe and secure by default because the shell never has the chance to do horrible things with user input.

What's more, while Julia's shell mini-language resembles POSIX syntax on a surface level, it is actually much saner and safer. It's very easy to convert a working Bash script to Julia, but the result will usually be safer in the end, which you can't say in most languages! For example, in a Bash script, you should not really do this:

```bash
link README.rst $filename
```

You should always put double quotes around the variable, because otherwise it will be expanded into multiple arguments on whitespace. However, in Julia, interpolated strings are never expanded in this way. Some things are expanded, however: iterables

In [5]:
`echo $(1:10)`

`echo 1 2 3 4 5 6 7 8 9 10`

As you can see, this is expanded by Julia before the command is even run. These can also combine with other elements to make Cartesian products in a way similar to how brace expansion works in the shell:

In [6]:
`./file$(1:10)`

`./file1 ./file2 ./file3 ./file4 ./file5 ./file6 ./file7 ./file8 ./file9 ./file10`

In [7]:
words = ["foo", "bar", "baz"]
numbers = 1:3
`$words$numbers`

`foo1 foo2 foo3 bar1 bar2 bar3 baz1 baz2 baz3`

As seen in some of these examples, using a `$()` inside of a command doesn't do process substitution as in the shell, it does, uh, "Julia substitution," as it would in a Julia string--aside from the expansion of iterables.

Julia has some other nice, logical features around commands. For example, when a process exits with a non-zero exit code in Bash, the script just tries to keep going and do who know's what. Same goes for starting processes in most other languages. That's just silly, and Julia knows it.

In [8]:
run(`link README.rst $filename`)

link: cannot create link 'foo.txt' to 'README.rst': File exists


ErrorException: failed process: Process(`link README.rst foo.txt`, ProcessExited(1)) [1]

That's right: Finished processes raise an error when there is a non-zero exit status in the general case. Why doesn't every other language do this by default? No idea. There are cases where you don't want this, like if you're using `grep`, for example. `grep` exits 1 if no matches were found, which isn't exactly an error.

You can avoid it by passing additional arguments to the `Cmd` constructor.

In [9]:
run(Cmd(`link README.rst $filename`, ignorestatus=true))

link: cannot create link 'foo.txt' to 'README.rst': File exists


Process(`link README.rst foo.txt`, ProcessExited(1))

So the error message still goes to stderr, because it's from the process itself, but it prevents a non-zero exit status from throwing an error.

Another nice feature that show that the Julia developers "get it" when it comes to processes, is that basically any function that can be applied to a file can be applied to a command literal.

In [10]:
readlines(`ls`)

6-element Array{String,1}:
 "CLI.ipynb"       
 "files.ipynb"     
 "filesystem.ipynb"
 "foo.txt"         
 "processes.ipynb" 
 "README.rst"      

In [11]:
open(`tr a-z A-Z`, "w", stdout) do io
    println(io, "foo")
end

FOO


Julia also supports pipelines, of course, but not with the pipe operator, `|`. Instead, one uses the `pipeline` function, which is also useful if you want to do more complex IO things. Rather than cover all this here, I will once again direct the reader to the [documentation](https://docs.julialang.org/en/v1/manual/running-external-programs/#Pipelines-1), where it is all laied out very clearly.

Word of warning to the reader: while it's wonderful that it's so easy and safe to work with processes in Julia, keep in mind that starting a process is very expensive for the OS relative t executing code in the current process. Particularly inside of hot loops, You should look for a way to do what you need directly in Julia first, and only resort to calling process when there is no apparent way to do the needful natively. It is so much slower.

One place where someone with a background writing shell scripts in other languages, but not as much experience in other languages might be tempted to use for string filtering utilities in coreutils--sed, grep, awk, etc. This would usually be a no-no, so the next section will provide a quick introduction about how to do the kinds of things you frequently do with those tools using Julia's regular expressions.

In [12]:
rm("foo.txt")